# Gather Drones

This prints out a list of all drone types used in the scenarios together with the performance data that is available in `BlueSky`.

First, lets import everything we need:

In [1]:
import os
import glob
import urllib
import json

import pandas as pd

from IPython.display import HTML

## Scan folders

Then we define the folder structure and scan these for `.scn` files that contain the operator used to create aircrafts in `BlueSky`:

In [2]:
SEARCH_OPERATOR = 'CRE'

scenario_files = os.path.join('..', '..', 'scenario', '**', '*.scn')
lines = []
for scenario_file in glob.glob(scenario_files, recursive=True):
    with open(scenario_file) as f:
        lines.extend([line for line in f.readlines() if SEARCH_OPERATOR in line])
print(f'Found {len(lines)} aircraft definitions at "{scenario_files}"')

Found 6 aircraft definitions at "..\..\scenario\**\*.scn"


## Extract aircrafts

Each of these lines is then split according to the command definition to extract the names and types of aircraft used:

In [3]:
aircraft_types = set()
for line in lines:
    type = line.strip().split()[3]
    aircraft_types.add(type)
print(f'Found {len(aircraft_types)} unique aircraft types in all exercises: {", ".join(aircraft_types)}')

Found 2 unique aircraft types in all exercises: M600, EC35


## Attach Performance data

All aircraft that were found are then checked against the current version of the `BlueSky` performance table, filtering it down to the relevant types. The object is then converted into a `DataFrame`, mostly to be able to view it more easily later on:

In [4]:
JSON_URL = 'https://raw.githubusercontent.com/USEPE-SesarJU/bluesky/master/data/performance/OpenAP/rotor/aircraft.json'
with urllib.request.urlopen(JSON_URL) as url:
    aircraft_data = json.loads(url.read().decode())

aircraft_data = {key: aircraft_data[key] for key in aircraft_types}
df = pd.DataFrame.from_dict(aircraft_data, orient='index').drop(['n_engines', 'engine_type'], axis=1)

aircraft_found = set(aircraft_data.keys())
if aircraft_types == aircraft_found:
    print('All aircrafts were found in the performance tables.')
else:
    print(f'No performance data for {", ".join(aircraft_types.difference(aircraft_found))}.')

All aircrafts were found in the performance tables.


## Output

Finally, the `DataFrame` is converted to an `HTML` output and the names are enhanced with a link (hopefully) leading to more data about the aircraft:

In [5]:
SEARCH_URL = 'https://duckduckgo.com/?q=!ducky'
df['name'] = df['name'].apply(lambda n: f'<a href="{SEARCH_URL}+{urllib.parse.quote_plus(n)}">{n}</a>')
HTML(df.to_html(escape=False))

,name,mtow,oew,mfc,engines,envelop
M600,DJI Matrice 600,15.1,9.1,0,"[[Motor-6010-1, 0.482], [Motor-6010-2, 0.482], [Motor-6010-3, 0.482], [Motor-6010-4, 0.482], [Motor-6010-5, 0.482], [Motor-6010-6, 0.482]]","{'v_min': -18, 'v_max': 18, 'vs_min': -5, 'vs_max': 5, 'h_max': 2500, 'd_range_max': 10}"
EC35,Eurocopter EC135,2835.0,1490.0,0,"[[PW206B, 463], [Arrius-2B2, 472]]","{'v_min': -20, 'v_max': 71.95, 'vs_min': -7.62, 'vs_max': 7.62, 'h_max': 6096, 'd_range_max': 665}"
